In [30]:
import json
import codecs
import numpy as np
import pandas as pd

project_name = 16069
with codecs.open(f"ALGORITM/data_for_algoritm_{project_name}.json", "r", "utf-8") as f:
    dfa = json.load(f)

df = pd.DataFrame.from_dict(dfa, orient='index')
boxes = pd.DataFrame(df.iloc[0].iloc[0])
cargo_space = df.iloc[0].iloc[4]
boxes_df = pd.DataFrame()
boxes_df[['color', 'title', 'article']] = boxes['info'].apply(lambda x: pd.Series([x['color'], x['title'][:45], x['article']]))
boxes_df['mass'] = boxes['mass']
boxes_df[['width', 'height', 'length']] = boxes['size'].apply(lambda x: pd.Series([x['width'], x['height'], x['length']]))
boxes_df['count'] = boxes['count']
boxes_df[['margin_width', 'margin_height', 'margin_length']] = boxes['margin'].apply(lambda x: pd.Series([x['width'], x['height'], x['length']]))
boxes_df['turnover'] = boxes['turnover']
boxes_df['stacking'] = boxes['stacking']
boxes_df['stacking_limit'] = boxes['stacking_limit']
cargoes = boxes_df.to_dict('records')
cargo_space_id = cargo_space['id']
cargo_space_size = cargo_space['size']
print(cargo_space_id)
boxes_df

266


,color,title,article,mass,width,height,length,count,margin_width,margin_height,margin_length,turnover,stacking,stacking_limit
0,#7d2ac1,опал,,300.0,3300,300,3300,7,0,0,0,True,True,0
1,#b243d3,опалубка 4,,80.0,300,300,3300,76,0,0,0,True,True,0
2,#2ae814,опал,,138.0,800,100,3300,45,0,0,0,True,True,0
3,#5e49bc,опал,,138.0,900,100,3300,40,0,0,0,True,True,0
4,#a51712,опалубка3,,170.0,1300,100,3300,24,0,0,0,True,True,0
5,#fc588f,ОПАЛУБКА2,,93.0,500,100,3300,19,0,0,0,True,True,0
6,#e1ea62,опалубка,,165.0,1200,100,3300,285,0,0,0,True,True,0


In [31]:
import random

def choose_random_box(boxes_df):
    dist = []
    running_total = 0
    for quantity in boxes_df['count']:
        running_total += quantity
        dist.append(running_total)
    r = random.randint(1, sum(boxes_df['count']))
    for i, total in enumerate(dist):
        if r <= total:
            box = i
            break
    return box

In [32]:
boxes_volume = sum(boxes_df['width'] * boxes_df['length'] * boxes_df['height'] * boxes_df['count'])
cargo_space_volume = cargo_space_size['width'] * cargo_space_size['length'] * cargo_space_size['height']
volume_ratio = boxes_volume / cargo_space_volume
from math import ceil
num_of_groups = ceil(volume_ratio)
if num_of_groups - volume_ratio < 0.2:
    num_of_groups += 1
boxes_list = boxes_df.copy()
random_boxes_groups = [pd.DataFrame(columns=boxes_list.columns)] * num_of_groups

In [33]:
iterations = 0
max_iterations = sum(boxes_list['count'])
remaining_volume = [cargo_space_volume] * num_of_groups

while iterations < max_iterations and not boxes_list.empty:
    for j in range(num_of_groups):
        if iterations >= max_iterations or boxes_list.empty:
            break
        box_index = choose_random_box(boxes_list)
        box_volume = boxes_list.loc[box_index, 'width'] * boxes_list.loc[box_index, 'length'] * boxes_list.loc[box_index, 'height']
        if box_volume < remaining_volume[j]:
            remaining_volume[j] -= box_volume
        else:
            break
        box_title = boxes_list.loc[box_index, 'title']
        if box_title in random_boxes_groups[j]['title'].values:
            random_boxes_groups[j].loc[random_boxes_groups[j]['title'] == box_title, 'count'] += 1
        else:
            random_boxes_groups[j] = pd.concat([random_boxes_groups[j], boxes_list.iloc[box_index].to_frame().T], ignore_index=True)
            random_boxes_groups[j].loc[random_boxes_groups[j]['title'] == box_title, 'count'] = 1
        if boxes_list.loc[box_index, 'count'] > 1:
            boxes_list.loc[box_index, 'count'] -= 1
        else:
            boxes_list.drop(box_index, axis = 0, inplace=True)
            boxes_list = boxes_list.reset_index(drop=True)
        iterations += 1

In [34]:
for i in range(num_of_groups):
    print(sum(random_boxes_groups[i]['width'] * random_boxes_groups[i]['length'] * random_boxes_groups[i]['height'] * random_boxes_groups[i]['count'])/cargo_space_volume)

0.6919646895864706
0.6839141308710601
0.6708798929508718


In [35]:
boxes_list = boxes_df.copy()
boxes_groups = [pd.DataFrame(columns=boxes_list.columns)] * num_of_groups
boxes_list['volume'] = boxes_list['width'] * boxes_list['length'] * boxes_list['height']
boxes_list = boxes_list.sort_values(by=['stacking', 'turnover', 'volume'])
boxes_list.drop('volume', axis=1, inplace=True)
boxes_list = boxes_list.reset_index(drop=True)

In [36]:
boxes_list

,color,title,article,mass,width,height,length,count,margin_width,margin_height,margin_length,turnover,stacking,stacking_limit
0,#fc588f,ОПАЛУБКА2,,93.0,500,100,3300,19,0,0,0,True,True,0
1,#2ae814,опал,,138.0,800,100,3300,45,0,0,0,True,True,0
2,#b243d3,опалубка 4,,80.0,300,300,3300,76,0,0,0,True,True,0
3,#5e49bc,опал,,138.0,900,100,3300,40,0,0,0,True,True,0
4,#e1ea62,опалубка,,165.0,1200,100,3300,285,0,0,0,True,True,0
5,#a51712,опалубка3,,170.0,1300,100,3300,24,0,0,0,True,True,0
6,#7d2ac1,опал,,300.0,3300,300,3300,7,0,0,0,True,True,0


In [37]:
iterations = 0
max_iterations = sum(boxes_list['count'])
remaining_volume = [cargo_space_volume] * num_of_groups

while iterations < max_iterations and not boxes_list.empty:
    for j in range(num_of_groups) if iterations%2 else range(num_of_groups-1, -1, -1):
        if iterations >= max_iterations or boxes_list.empty:
            break
        box_volume = boxes_list.loc[len(boxes_list)-1, 'width'] * boxes_list.loc[len(boxes_list)-1, 'length'] * boxes_list.loc[len(boxes_list)-1, 'height']
        if box_volume < remaining_volume[j]:
            remaining_volume[j] -= box_volume
        else:
            break
        box_title = boxes_list.loc[len(boxes_list)-1, 'title']
        if box_title in boxes_groups[j]['title'].values:
            boxes_groups[j].loc[boxes_groups[j]['title'] == box_title, 'count'] += 1
        else:
            boxes_groups[j] = pd.concat([boxes_groups[j], boxes_list.iloc[len(boxes_list)-1].to_frame().T], ignore_index=True)
            boxes_groups[j].loc[boxes_groups[j]['title'] == box_title, 'count'] = 1
        if boxes_list.loc[len(boxes_list)-1, 'count'] > 1:
            boxes_list.loc[len(boxes_list)-1, 'count'] -= 1
        else:
            boxes_list.drop(len(boxes_list)-1, axis = 0, inplace=True)
            boxes_list = boxes_list.reset_index(drop=True)
        iterations += 1

In [38]:
for i in range(num_of_groups):
    print(sum(boxes_groups[i]['width'] * boxes_groups[i]['length'] * boxes_groups[i]['height'] * boxes_groups[i]['count'])/cargo_space_volume)

1.68525029109259
1.7511882005711898
1.784540515249319


In [39]:
dict_boxes_groups = [dict] * num_of_groups
dict_random_boxes_groups = [dict] * num_of_groups
for j in range(num_of_groups):
    dict_boxes_groups[j] = boxes_groups[j].to_dict('records')
    dict_random_boxes_groups[j] = random_boxes_groups[j].to_dict('records')

In [40]:
import requests

url = "https://back.glsystem.net/api/v1/auth/login/"
data = {
    'username': 'm2213635@edu.misis.ru',
    'password': 'Bgp6XvJCnD'
}

response = requests.post(url, data=data)
print(response.text)
token = response.text[17:57]

{"access_token":"75e3e611641a48d4074f003c4ab94516d66697b3","refresh_token":"c0cec5b0931d595a4166cb50a978ae6d8fd1b5ca","token_type":"Bearer","access_token_expires":0,"refresh_token_expires":0}


In [41]:
project_title = f'random_algoritm_{project_name}'
url = "https://back.glsystem.net/api/v1/project/"
headers = {
    'authorization': 'Bearer ' + token
}
data = {
    'title': project_title
}

response = requests.post(url, headers=headers, data=data)
print(response.text)
project_id = response.text[6:10]

{"id":6513,"title":"random_algoritm_16069","created_at":"2023-07-24T14:51:33.218754","updated_at":"2023-07-24T14:51:33.299925","demo":true,"is_calculating":false,"absolute_url":"/random-algoritm-16069-6513"}


In [42]:
url = "https://back.glsystem.net/api/v1/calculation/"
headers = {
    'content-type': 'application/json;charset=UTF-8',
    'Authorization': 'Bearer ' + token
}
dict = {
    "project": project_id,
    "input_data": {
    "cargo_spaces": [
        cargo_space_id
    ] * num_of_groups,
    "groups": [
    {
        "title": "string",
        "pallet": 0,
        "cargoes": dict_random_boxes_groups[i],
        "sort": 1
    } for i in range(num_of_groups)
    ],
    "userSort": True
    },
    "status": "new",
    "external_api": True
}
    
data = json.dumps(dict)

response = requests.post(url, headers=headers, data=data)
print(response.text)
calculation_id = response.text[6:11]

{"id":30988,"project":6513,"status":"in_queue","callback_url":"","external_api":true}


In [ ]:
project_title = f'optimized_algoritm_{project_name}'
url = "https://back.glsystem.net/api/v1/project/"
headers = {
    'authorization': 'Bearer ' + token
}
data = {
    'title': project_title
}

response = requests.post(url, headers=headers, data=data)
print(response.text)
project_id = response.text[6:10]

In [ ]:
url = "https://back.glsystem.net/api/v1/calculation/"
headers = {
    'content-type': 'application/json;charset=UTF-8',
    'Authorization': 'Bearer ' + token
}
dict = {
    "project": project_id,
    "input_data": {
    "cargo_spaces": [
        cargo_space_id
    ] * num_of_groups,
    "groups": [
    {
        "title": "string",
        "pallet": 0,
        "cargoes": dict_boxes_groups[i],
        "sort": 1
    } for i in range(num_of_groups)
    ],
    "userSort": True
    },
    "status": "new",
    "external_api": True
}
    
data = json.dumps(dict)

response = requests.post(url, headers=headers, data=data)
print(response.text)
calculation_id = response.text[6:11]